In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
df=pd.read_csv(r"B:\project\newdataset\chunk_11.csv")
df = df.drop(columns=['Unnamed: 0'])
df

,text,generated
0,The election of a president is a very serious ...,0.0
1,The founding fathers established the Electoral...,0.0
2,"Dear Senator,\n\nConcerning the topic of the m...",0.0
3,How can you build self esteem. Selfesteem come...,0.0
4,"Dear Principal,\n\nI wrote this letter Te you ...",0.0
...,...,...
17983,"Well, I think that everybody does not Have to ...",0.0
17984,"You're running down the field, lacrosse stick ...",0.0
17985,Cars are a wonderful thing. They are perhaps o...,0.0
17986,The author suggest that studying genus is a wo...,0.0


In [2]:
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
essays=df['text'].tolist()
tokens=tokenizer(essays,return_tensors='pt',padding=True,truncation=True)

In [3]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
input_ids_size = tokens['input_ids'].size()
attention_mask_size = tokens['attention_mask'].size()
print("Input IDs size:", input_ids_size)
print("Attention Mask size:", attention_mask_size)
# Assuming 'tokens' is the result of tokenizer
unique_tokens = set(token.item() for seq in tokens['input_ids'] for token in seq)
vocabulary_size = len(unique_tokens)

print("Vocabulary Size:", vocabulary_size)
# Assuming 'input_ids' is the first element in xtrain
max_sequence_length_x = max(len(seq) for seq in xtrain)

# Print or use the max_sequence_length_x in your pad_sequences and Embedding layer
print("Max Sequence Length:", max_sequence_length_x)


Input IDs size: torch.Size([17988, 1930])
Attention Mask size: torch.Size([17988, 1930])
Vocabulary Size: 32975
Max Sequence Length: 1930


In [4]:
model=Sequential()
input_length=1930
xtrain_np=xtrain.numpy()
model.add(Embedding(input_dim=52000,output_dim=100,input_length=input_length))
model.add(LSTM(64))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xtrain_np,ytrain,epochs=5,batch_size=32)



Epoch 1/5


450/450 [==============================] - 310s 683ms/step - loss: 0.0129 - accuracy: 1.0000
Epoch 2/5
450/450 [==============================] - 296s 658ms/step - loss: 3.9329e-05 - accuracy: 1.0000
Epoch 3/5
450/450 [==============================] - 255s 567ms/step - loss: 2.0347e-05 - accuracy: 1.0000
Epoch 4/5
450/450 [==============================] - 273s 608ms/step - loss: 1.2620e-05 - accuracy: 1.0000
Epoch 5/5
450/450 [==============================] - 248s 551ms/step - loss: 8.5141e-06 - accuracy: 1.0000


In [5]:
xtest_np=xtest.numpy()
model.evaluate(xtest_np,ytest)

113/113 [==============================] - 19s 163ms/step - loss: 7.0852e-06 - accuracy: 1.0000


[7.085170636855764e-06, 1.0]

In [6]:
model.save("newmodel11.keras")